### Dependencias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from scipy import stats
from sklearn.preprocessing import KBinsDiscretizer


from auxiliares import freq, normalizar
import warnings
warnings.filterwarnings('ignore')
cf.go_offline()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Lectura de datos

In [2]:
ruta = '/home/guadalupe/analisis/Datos/datos'

In [3]:
train = pd.read_parquet(os.path.join(ruta,'apptrain.parquet'))
valid = pd.read_parquet(os.path.join(ruta,'appvalid.parquet'))
train.shape, valid.shape

((215257, 122), (92254, 121))

In [ ]:
buro = pd.read_parquet(os.path.join(ruta,'buro.parquet'))
buro.shape

(1716428, 17)

In [ ]:
train = pd.merge(train,buro, on ='application_id')
train.shape

In [ ]:
#credit_card = pd.read_parquet(os.path.join(ruta,'credit_card.parquet'))
#credit_card.shape

In [ ]:
train.sample(5)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,application_id
117380,0,Cash loans,F,N,Y,1,157500.0,518562.0,22099.5,463500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.018634,-11946,-727,-5126.0,-163,NaN,1,1,0,1,0,0,Sales staff,3.0,2,2,FRIDAY,16,0,0,0,1,1,0,Self-employed,NaN,0.356108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,5.0,0.0,5.0,0.0,-514.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,A000365635
55870,0,Cash loans,M,N,Y,0,180000.0,1256400.0,44644.5,900000.0,Children,Working,Secondary / secondary special,Married,House / apartment,0.025164,-20457,-540,-10968.0,-3941,NaN,1,1,0,1,0,0,Drivers,2.0,2,2,THURSDAY,11,0,0,0,0,1,1,Trade: type 7,NaN,0.650034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,0.0,1.0,0.0,-210.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,A000365473
41759,0,Cash loans,M,N,Y,0,171000.0,918468.0,30483.0,697500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.011657,-20681,-2513,-1873.0,-3702,NaN,1,1,0,1,0,0,Security staff,2.0,1,1,MONDAY,13,0,0,0,0,0,0,Business Entity Type 3,NaN,0.209170,0.728141,0.0082,0.0914,0.9786,0.7076,0.0042,0.00,0.069,0.1667,NaN,0.0190,NaN,0.0757,NaN,0.0630,0.0084,0.0948,0.9786,0.7190,0.0042,0.0000,0.069,0.1667,NaN,0.0194,NaN,0.0788,NaN,0.0667,0.0083,0.0914,0.9786,0.7115,0.0042,0.00,0.069,0.1667,NaN,0.0193,NaN,0.0770,NaN,0.0643,reg oper account,block of flats,0.0755,"Stone, brick",No,3.0,2.0,3.0,2.0,-1642.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0

In [ ]:
#train.columns.tolist()

### Caracterización de las variables

In [ ]:
varc = ['AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE','DAYS_BIRTH','DAYS_EMPLOYED',
 'DAYS_REGISTRATION','OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_YEAR']
vard = ['NAME_CONTRACT_TYPE','CODE_GENDER','NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE','FLAG_MOBIL','FLAG_CONT_MOBILE','OCCUPATION_TYPE','REG_REGION_NOT_LIVE_REGION',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3','FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',]
vart = ['TARGET']
um = ['application_id']

In [ ]:
train = train[um+vart+varc+vard]
train.shape

(215257, 50)